In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

c:\Users\alber\anaconda3\envs\env_gen_ai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# Carrega o modelo e o tokenizer do Hugging Face
#modelo_reranker = "cross-encoder/ms-marco-MiniLM-L-6-v2"
modelo_reranker = "cross-encoder/ms-marco-electra-base"
tokenizer = AutoTokenizer.from_pretrained(modelo_reranker)
modelo = AutoModelForSequenceClassification.from_pretrained(modelo_reranker)

c:\Users\alber\anaconda3\envs\env_gen_ai\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alber\.cache\huggingface\hub\models--cross-encoder--ms-marco-electra-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [14]:

# Função para pontuar cada documento com base na consulta
def pontuar_documentos(consulta, documentos):
    pontuacoes = []
    for documento in documentos:
        entradas = tokenizer(consulta, documento, return_tensors="pt", truncation=True, padding=True)
        pontuacao = modelo(**entradas).logits[0].item()
        pontuacoes.append((documento, pontuacao))
    
    # Ordena os documentos pela pontuação em ordem decrescente (mais relevante primeiro)
    documentos_ordenados = sorted(pontuacoes, key=lambda x: x[1], reverse=True)
    return documentos_ordenados

In [18]:
# Consulta
consulta = "Quais são as capitais dos principais países da Europa e suas principais características culturais e históricas?"

# Documentos mais longos
documentos = [
    # Documentos relacionados
    """Paris, capital da França, é conhecida por sua rica história, arquitetura e cultura.
    A cidade é um dos principais centros de arte e moda do mundo, e possui marcos famosos como a Torre Eiffel e o Louvre.""",
    
    """Londres é a capital da Inglaterra e do Reino Unido, e tem uma história que remonta a mais de dois milênios.
    A cidade é um dos principais centros financeiros do mundo e é famosa por pontos turísticos como o Big Ben, a London Eye e o Palácio de Buckingham.""",
    
    """Madri, capital da Espanha, é conhecida por sua vida vibrante e por abrigar algumas das melhores coleções de arte da Europa, como o Museu do Prado.
    A cidade também é famosa por sua arquitetura histórica e cultura rica, incluindo tradições como o flamenco e a culinária espanhola.""",
    
    """Berlim, a capital da Alemanha, é uma cidade com uma história complexa, especialmente relacionada à Segunda Guerra Mundial e à Guerra Fria.
    A cidade é conhecida por sua diversidade cultural, por seus museus e pela arte de rua. Também abriga o famoso Portão de Brandemburgo.""",
    
    """Roma é a capital da Itália e uma cidade com uma história extensa e profunda. Foi o centro do Império Romano e ainda preserva monumentos antigos como o Coliseu.
    Roma também é conhecida por seu papel na religião católica e abriga o Vaticano.""",
    
    """Lisboa, capital de Portugal, é famosa por suas colinas, seu bairro histórico Alfama e o clima mediterrâneo.
    A cidade combina uma arquitetura colorida, música de fado e uma longa história de navegações e descobertas.""",

    # Documentos não relacionados
    """O sistema solar é composto por oito planetas principais, cada um com suas características únicas. 
    Os planetas são Mercúrio, Vênus, Terra, Marte, Júpiter, Saturno, Urano e Netuno.""",

    """A fotografia é uma forma de arte que captura momentos no tempo. 
    Ela evoluiu com o passar dos anos, desde o filme até as câmeras digitais, e é amplamente utilizada para fins pessoais e profissionais.""",

    """Python é uma linguagem de programação popular e versátil, usada em diversos campos como ciência de dados, desenvolvimento web e automação.
    Ela é conhecida por sua sintaxe simples e comunidade ativa.""",
    
    """A Amazônia é a maior floresta tropical do mundo e desempenha um papel importante na regulação do clima global. 
    Ela abriga uma biodiversidade incrível e é fundamental para o ecossistema terrestre.""",
    
    """A evolução das espécies foi um conceito desenvolvido por Charles Darwin. 
    Esse processo explica como os seres vivos mudam e se adaptam ao longo de gerações.""",
]


In [19]:
# Executa a função de reranking
documentos_ordenados = pontuar_documentos(consulta, documentos)

In [20]:
# Exibe os resultados ordenados
print("Documentos ordenados por relevância:")
for doc, score in documentos_ordenados:
    print(f"Pontuação: {score:.4f} | Documento: {doc}")

Documentos ordenados por relevância:
Pontuação: 2.2512 | Documento: Madri, capital da Espanha, é conhecida por sua vida vibrante e por abrigar algumas das melhores coleções de arte da Europa, como o Museu do Prado.
    A cidade também é famosa por sua arquitetura histórica e cultura rica, incluindo tradições como o flamenco e a culinária espanhola.
Pontuação: 1.8982 | Documento: O sistema solar é composto por oito planetas principais, cada um com suas características únicas. 
    Os planetas são Mercúrio, Vênus, Terra, Marte, Júpiter, Saturno, Urano e Netuno.
Pontuação: 1.4346 | Documento: Paris, capital da França, é conhecida por sua rica história, arquitetura e cultura.
    A cidade é um dos principais centros de arte e moda do mundo, e possui marcos famosos como a Torre Eiffel e o Louvre.
Pontuação: -2.3207 | Documento: Londres é a capital da Inglaterra e do Reino Unido, e tem uma história que remonta a mais de dois milênios.
    A cidade é um dos principais centros financeiros do m